<a href="https://colab.research.google.com/github/travislatchman/Neuroimage-Registration-and-Synthesis/blob/main/Registration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Dependencies Installation

In [ ]:
!pip install nilearn
!pip install nibabel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 3.0.2
    Uninstalling nibabel-3.0.2:
      Successfully uninstalled nibabel-3.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install antspyx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 8.4 MB/s eta 0:00:00


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting
import ants

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_dir = "/content/drive/MyDrive/Project 2 Neuroimage Synthesis/MedIA_Project2_stripped"
output_dir = "/content/drive/MyDrive/Project 2 Neuroimage Synthesis/MedIA_Project2_registered"

In [ ]:
if os.path.exists(input_dir):
    print("Directory exists!")
else:
    print("Directory does not exist!")

Directory exists!


In [ ]:
if os.path.exists(output_dir):
    print("Directory exists!")
else:
    print("Directory does not exist!")

Directory exists!


# Registration

In [ ]:
for i in range(1, 201):
    # skip patient 163, doesn't have a ADC file
    if i == 163:
      continue

    patient_folder = f"{i:03d}"
    input_folder = os.path.join(input_dir, patient_folder)
    output_folder = os.path.join(output_dir, patient_folder)

    # Load the skull-stripped images
    t1w_img = ants.image_read(os.path.join(input_folder, "T1w_1mm.nii.gz"))
    t2w_img = ants.image_read(os.path.join(input_folder, "T2w_1mm_noalign.nii.gz"))
    fa_img = ants.image_read(os.path.join(input_folder, "FA_deformed.nii.gz"))
    adc_img = ants.image_read(os.path.join(input_folder, "ADC_deformed.nii.gz"))

    # Resample FA and ADC images to 1mm isotropic resolution
    fa_img_resampled = ants.resample_image(fa_img, t1w_img.shape, use_voxels=True, interp_type=1)
    adc_img_resampled = ants.resample_image(adc_img, t1w_img.shape, use_voxels=True, interp_type=1)

    # Rigid registration of T2w, FA, and ADC images to T1w image
    t2w_rigid = ants.registration(fixed=t1w_img, moving=t2w_img, type_of_transform="Rigid")
    fa_rigid = ants.registration(fixed=t1w_img, moving=fa_img_resampled, type_of_transform="Rigid")
    adc_rigid = ants.registration(fixed=t1w_img, moving=adc_img_resampled, type_of_transform="Rigid")

    t2w_rigid_img = t2w_rigid["warpedmovout"]
    fa_rigid_img = fa_rigid["warpedmovout"]
    adc_rigid_img = adc_rigid["warpedmovout"]

    # Non-rigid registration of FA and ADC images to T1w image
    fa_nrigid = ants.registration(fixed=t1w_img, moving=fa_rigid_img, type_of_transform="SyN")
    adc_nrigid = ants.registration(fixed=t1w_img, moving=adc_rigid_img, type_of_transform="SyN")

    fa_nrigid_img = fa_nrigid["warpedmovout"]
    adc_nrigid_img = adc_nrigid["warpedmovout"]

    # Resample registered FA and ADC images back to 1.25mm isotropic resolution
    fa_nrigid_img_125 = ants.resample_image(fa_nrigid_img, fa_img.shape, use_voxels=True, interp_type=1)
    adc_nrigid_img_125 = ants.resample_image(adc_nrigid_img, adc_img.shape, use_voxels=True, interp_type=1)

    # Create output directory and save the registered images
    os.makedirs(output_folder, exist_ok=True)


    ants.image_write(t1w_img, os.path.join(output_folder, "T1w_1mm.nii.gz"))
    ants.image_write(t2w_rigid_img, os.path.join(output_folder, "T2w_1mm_noalign.nii.gz"))
    ants.image_write(fa_nrigid_img_125, os.path.join(output_folder, "FA_deformed.nii.gz"))
    ants.image_write(adc_nrigid_img_125, os.path.join(output_folder, "ADC_deformed.nii.gz"))
